In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2
from torchvision import transforms

torch.cuda.is_available()

True

# Problem parameters

In [2]:
test = 4  # 1: MbV2, 2: MbV2_CA, 3: MbV2_CA_Reduced, 4: MbV2_Reduced

DB_used = 'ISIC2019' # 'HAM10000', 'ISIC2019', 'PH2'

save_splited_images = False

In [3]:
# OTHER PARAMS

inverted_residual_setting_v0 = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ] #ORIGINAL

inverted_residual_setting_vT3 = [
        # t, c, n, s
        [1, 16, 1, 1],
        [4, 24, 1, 2],
        [4, 32, 1, 2],
        [4, 64, 1, 2],
        [4, 96, 1, 1],
        [4, 160, 1, 2],
        [4, 320, 1, 1],
    ]

lr = 1e-4
n_epoch = 20
batch_size = 40

if test == 1:
    Test = 'MbV2'
    CoordAtt = False
    inverted_residual_setting = inverted_residual_setting_v0
elif test == 2:
    Test = 'MbV2_CA'
    CoordAtt = True
    inverted_residual_setting = inverted_residual_setting_v0
elif test == 3:
    Test = 'MbV2_CA_Reduced'
    CoordAtt = True
    inverted_residual_setting = inverted_residual_setting_vT3
elif test == 4:
    Test = 'MbV2_Reduced'
    CoordAtt = False
    inverted_residual_setting = inverted_residual_setting_vT3

if DB_used == 'HAM10000':
    dataset_dir = os.path.join(config.DATASET_DIR,'HAM10000_splited')
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    transform_flip = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()
    ])
elif DB_used == 'ISIC2019':
    dataset_dir = os.path.join(config.DATASET_DIR,'ISIC2019_splited')
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    transform_flip = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()
    ])
elif DB_used == 'PH2':
    dataset_dir = os.path.join(config.DATASET_DIR,'PH2_up_splited')
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256))
    ])
    transform_flip = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()
    ])
    lr = 5e-5
    batch_size = 6


CIFAR_dir = os.path.join(config.RESULT_DIR,'weights',Test,'features.pth')
log_dir = os.path.join(config.RESULT_DIR,'log',Test,DB_used) 
model_dir = os.path.join(config.RESULT_DIR,'weights',Test,DB_used,'model.pth')

labels = [0, 1]
split_ratio = [0.8,0.1,0.1] # train, val, test
Weighted_sampling = True

criterion = torch.nn.CrossEntropyLoss()

# Dataset

In [4]:
if save_splited_images:
    dataset = Derma(dataset_dir,labels=labels,transform=transform_flip)
    train_set, val_set, test_set = dataset.split_rand(split_ratio)
else:
    train_set = Derma(os.path.join(dataset_dir,'train'),labels=labels,transform=transform_flip)
    val_set = Derma(os.path.join(dataset_dir,'val'),labels=labels,transform=transform)
    test_set = Derma(os.path.join(dataset_dir,'test'),labels=labels,transform=transform)

In [5]:
# Weighted sampling
if Weighted_sampling:
    from derma.dataset import get_samples_weight
    train_sampler, train_weights = get_samples_weight(train_set)
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0, sampler=train_sampler, shuffle=False)
    val_sampler, val_weights = get_samples_weight(val_set)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=0, sampler=val_sampler, shuffle=False)
    test_sampler, test_weights = get_samples_weight(test_set)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0, sampler=test_sampler, shuffle=False)
else:
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=0)


Samples per class: [3589 3646]
Weight per class: [0.00027863 0.00027427]
Samples per class: [483 421]
Weight per class: [0.00207039 0.0023753 ]
Samples per class: [450 455]
Weight per class: [0.00222222 0.0021978 ]


In [6]:
if save_splited_images:
    from PIL import Image
    from torchvision.transforms.functional import resize
    for folder in ['train','val','test']:
        names = dataset.getnames()
        y = dataset.get_labels()
        if folder == 'train':
            indexes = train_loader.dataset.indices
        elif folder == 'val':
            indexes = val_loader.dataset.indices
        elif folder == 'test':
            indexes = test_loader.dataset.indices
        path_base = os.path.join(dataset_dir,folder)
        for i in indexes:
            _, image_name = os.path.split(names[i])
            image = Image.open(names[i]).convert('RGB')
            image = resize(image,[256,256])
            image.save(os.path.join(path_base,str(int(y[i])),image_name))

# Model

In [7]:
if CoordAtt:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
else:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting) # standard MobileNetV2

In [8]:
# LOADIONG CIFAR WEIGHTS
features = torch.load(CIFAR_dir)
model.features.load_state_dict(features)

<All keys matched successfully>

# Training

In [9]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
tb_writer = SummaryWriter(log_dir=log_dir)

train(model, [train_loader,val_loader], optimizer, criterion, n_epoch, tb_writer)

100%|██████████| 20/20 [09:25<00:00, 28.29s/epoch, acc=0.8015, tls=0.4274]


0.418884444495906

In [10]:
# torch.save(model.features.state_dict(),model_dir) # Hay que guardar *todo*
torch.save(model.state_dict(),model_dir)

# Test

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.load_state_dict(torch.load(model_dir))
    model.to(device)
else:
    model.load_state_dict(torch.load(model_dir),map_location=torch.device('cpu'))

In [12]:
from derma.experiment import test_experiment
metrics_df = test_experiment(model,test_loader)
metrics_df.head()

,Sensitivity,Specificity,Precission,Recall,Accuracy
0,0.875,0.764706,0.636364,0.875,0.8
